In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext(appName="AudioScrobbler")

In [2]:
#load the data
rawuserartistdata = sc.textFile("user_artist_data.txt")

In [3]:
#check if maximum user and artist IDs are smaller than 2147483647
rawuserartistdata.map(lambda x: float(x.split()[0])).stats()

(count: 24296858, mean: 1947573.2653533542, stdev: 496000.544975, max: 2443548.0, min: 90.0)

In [4]:
rawuserartistdata.map(lambda x: float(x.split()[1])).stats()

(count: 24296858, mean: 1718704.0937569223, stdev: 2539389.04017, max: 10794401.0, min: 1.0)

In [5]:
# split data in 1:9 ratio for test and training respectively
#weights = [.1, .9]
#seed = 10000
#sample, test = rawuserartistdata.randomSplit(weights, seed)
#sample.cache()
rawuserartistdata.cache()

user_artist_data.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [6]:
rawartistdata=sc.textFile('artist_data.txt')
def splitlines(singlePair):
    words = singlePair.rsplit('\t')
    if len(words) != 2:
        return []
    else:
        try:
            return [(int(words[0]), words[1])]
        except:
            return []

artistByID = dict(rawartistdata.flatMap(lambda x: splitlines(x)).collect())

In [7]:
rawAliasdata = sc.textFile('artist_alias.txt')
artistAlias = rawAliasdata.flatMap(lambda x: splitlines(x)).collectAsMap()

In [8]:
from pyspark.mllib import recommendation
from pyspark.mllib.recommendation import *

bartistAlias = sc.broadcast(artistAlias)

def mapAlias(x):
    userID, artistID, count = map(lambda lineItem: int(lineItem), x.split())
    finalArtistID = bartistAlias.value.get(artistID)
    if finalArtistID is None:
        finalArtistID = artistID
    return Rating(userID, finalArtistID, count)

trainData = rawuserartistdata.map(lambda x: mapAlias(x))
trainData.cache()

NameError: name 'sample' is not defined

In [58]:
# training the ALS model with hyperparameters
model = ALS.trainImplicit(trainData, 10, 5, 0.01)

In [60]:
#recommendations for user id #2093760
testUserID = 2093760
artistByIDBroadcast = sc.broadcast(artistByID)

artistsForUser = (trainData
                  .filter(lambda observation: observation.user == testUserID)
                  .map(lambda observation: artistByIDBroadcast.value.get(observation.product))
                  .collect())
print (artistsForUser)

['The Saw Doctors', 'Xzibit']


In [39]:
recommendationsForUser=model.recommendProducts(testUserID, 5)
#print (recommendationsForUser)
for rating in recommendationsForUser:
    print (artistByIDBroadcast.value.get(rating.product), end='\',\' ')

[Rating(user=2093760, product=4267, rating=0.00563151477170487), Rating(user=2093760, product=3327, rating=0.004941007386155507), Rating(user=2093760, product=1001779, rating=0.0038544516499355993), Rating(user=2093760, product=1854, rating=0.003692966249263605), Rating(user=2093760, product=1233770, rating=0.0036272200856840163)]
Green Day',' Weezer',' Modest Mouse',' Linkin Park',' The Shins',' 

In [12]:
get_ipython().system(u'jupyter nbconvert --to python AudioScrobbler.ipynb')

[NbConvertApp] Converting notebook AudioScrobbler.ipynb to python
[NbConvertApp] Writing 2379 bytes to AudioScrobbler.py
